___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___

<h1><p style="text-align: center;">Data Analysis with Python <br>Project - 1</p><h1> - Traffic Police Stops <img src="https://docs.google.com/uc?id=17CPCwi3_VvzcS87TOsh4_U8eExOhL6Ki" class="img-fluid" alt="CLRSWY" width="200" height="100"> 

Does the ``gender`` of a driver have an impact on police behavior during a traffic stop? **In this chapter**, you will explore that question while practicing filtering, grouping, method chaining, Boolean math, string methods, and more!

***

## Examining traffic violations

Before comparing the violations being committed by each gender, you should examine the ``violations`` committed by all drivers to get a baseline understanding of the data.

In this exercise, you'll count the unique values in the ``violation`` column, and then separately express those counts as proportions.

> Before starting your work in this section **repeat the steps which you did in the previos chapter for preparing the data.** Continue to this chapter based on where you were in the end of the previous chapter.

In [1]:
import numpy as pd
import pandas as pd

In [3]:
ri = pd.read_csv('RI_cleaned.csv', low_memory=False)

In [4]:
ri = ri.drop(['county_name','county_fips','fine_grained_location','state'], axis=1)

In [5]:
ri.dropna(how='all', subset=['driver_gender'],inplace=True)

In [6]:
ri['is_arrested'] = ri['is_arrested'].astype('bool')

In [7]:
combined = ri['stop_date'] + ' ' + ri['stop_time']

In [8]:
ri['stop_datetime'] = pd.to_datetime(combined)

In [9]:
ri = ri.set_index(pd.DatetimeIndex(ri['stop_datetime']))

In [10]:
ri.drop('stop_datetime', axis = 1, inplace = True)

In [11]:
ri.columns

Index(['id', 'stop_date', 'stop_time', 'location_raw', 'police_department',
       'driver_gender', 'driver_age_raw', 'driver_age', 'driver_race_raw',
       'driver_race', 'violation_raw', 'violation', 'search_conducted',
       'search_type_raw', 'search_type', 'contraband_found', 'stop_outcome',
       'is_arrested', 'stop_duration', 'out_of_state', 'drugs_related_stop',
       'district'],
      dtype='object')

In [12]:
ri.dtypes

id                     object
stop_date              object
stop_time              object
location_raw           object
police_department      object
driver_gender          object
driver_age_raw        float64
driver_age            float64
driver_race_raw        object
driver_race            object
violation_raw          object
violation              object
search_conducted       object
search_type_raw        object
search_type            object
contraband_found         bool
stop_outcome           object
is_arrested              bool
stop_duration          object
out_of_state           object
drugs_related_stop       bool
district               object
dtype: object

In [13]:
ri.index

DatetimeIndex(['2005-01-02 01:55:00', '2005-01-02 20:30:00',
               '2005-01-04 12:55:00', '2005-01-06 01:30:00',
               '2005-01-12 08:05:00', '2005-01-18 08:15:00',
               '2005-01-18 17:13:00', '2005-01-23 23:15:00',
               '2005-01-24 20:32:00', '2005-02-09 03:05:00',
               ...
               '2015-12-31 22:10:00', '2015-12-31 22:14:00',
               '2015-12-31 22:26:00', '2015-12-31 22:38:00',
               '2015-12-31 22:39:00', '2015-12-31 22:46:00',
               '2015-12-31 22:47:00', '2015-12-31 23:08:00',
               '2015-12-31 23:44:00', '2015-12-31 23:48:00'],
              dtype='datetime64[ns]', name='stop_datetime', length=480584, freq=None)

In [14]:
ri.head()

,id,stop_date,stop_time,location_raw,police_department,driver_gender,driver_age_raw,driver_age,driver_race_raw,driver_race,...,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
stop_datetime,,,,,,,,,,,,,,,,,,,,,
2005-01-02 01:55:00,RI-2005-00001,2005-01-02,01:55,Zone K1,600,M,1985.0,20.0,W,White,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K1
2005-01-02 20:30:00,RI-2005-00002,2005-01-02,20:30,Zone X4,500,M,1987.0,18.0,W,White,...,False,NaN,NaN,False,Citation,False,16-30 Min,False,False,Zone X4
2005-01-04 12:55:00,RI-2005-00004,2005-01-04,12:55,Zone X4,500,M,1986.0,19.0,W,White,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2005-01-06 01:30:00,RI-2005-00005,2005-01-06,01:30,Zone X4,500,M,1978.0,27.0,B,Black,...,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2005-01-12 08:05:00,RI-2005-00006,2005-01-12,08:05,Zone X1,000,M,1973.0,32.0,B,Black,...,False,NaN,NaN,False,Citation,False,30+ Min,True,False,Zone X1


**INSTRUCTIONS**

*   Count the unique values in the ``violation`` column, to see what violations are being committed by all drivers.
*   Express the violation counts as proportions of the total.

In [15]:
ri['violation'].value_counts(dropna=False)

Speeding               268736
Moving violation        90228
Equipment               61250
Other                   24216
Registration/plates     19830
Seat belt               16324
Name: violation, dtype: int64

In [16]:
len(ri['violation'])

480584

In [17]:
ri['violation'].isnull().sum()

0

In [18]:
(ri['violation'].value_counts(dropna=False)/len(ri['violation'])) * 100

Speeding               55.918632
Moving violation       18.774658
Equipment              12.744910
Other                   5.038869
Registration/plates     4.126230
Seat belt               3.396701
Name: violation, dtype: float64

***

## Comparing violations by gender

The question we're trying to answer is whether male and female drivers tend to commit different types of traffic violations.

You'll first create a ``DataFrame`` for each gender, and then analyze the ``violations`` in each ``DataFrame`` separately.

**INSTRUCTIONS**

*   Create a ``DataFrame``, female, that only contains rows in which ``driver_gender`` is ``'F'``.
*   Create a ``DataFrame``, male, that only contains rows in which ``driver_gender`` is ``'M'``.
*   Count the ``violations`` committed by female drivers and express them as proportions.
*   Count the violations committed by male drivers and express them as proportions.

In [19]:
pd.set_option('display.max_columns', 50)

In [20]:
ri.sample(10)

,id,stop_date,stop_time,location_raw,police_department,driver_gender,driver_age_raw,driver_age,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district
stop_datetime,,,,,,,,,,,,,,,,,,,,,,
2013-08-15 16:01:00,RI-2013-27768,2013-08-15,16:01,Zone X3,200,M,1985.0,28.0,H,Hispanic,Speeding,Speeding,True,Incident to Arrest,Incident to Arrest,False,Arrest Driver,True,0-15 Min,False,False,Zone X3
2006-07-07 03:10:00,RI-2006-30510,2006-07-07,03:10,Zone K1,600,M,1989.0,17.0,B,Black,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K1
2011-05-04 20:31:00,RI-2011-13573,2011-05-04,20:31,Zone X3,200,M,1975.0,36.0,W,White,Equipment/Inspection Violation,Equipment,False,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone X3
2015-10-03 10:22:00,RI-2015-35798,2015-10-03,10:22,Zone X4,500,M,1958.0,57.0,B,Black,Equipment/Inspection Violation,Equipment,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4
2006-11-15 10:35:00,RI-2006-51155,2006-11-15,10:35,Zone X4,500,M,1959.0,47.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone X4
2013-07-11 07:35:00,RI-2013-23548,2013-07-11,07:35,Zone K3,300,F,1969.0,44.0,W,White,Other Traffic Violation,Moving violation,True,Probable Cause,Probable Cause,True,Warning,False,0-15 Min,False,False,Zone K3
2013-11-23 15:00:00,RI-2013-40245,2013-11-23,15:00,Zone X4,500,M,1990.0,23.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,16-30 Min,False,False,Zone X4
2006-12-16 02:21:00,RI-2006-56090,2006-12-16,02:21,Zone K3,300,M,1982.0,24.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,True,False,Zone K3
2015-04-27 09:25:00,RI-2015-13582,2015-04-27,09:25,Zone X3,200,M,1985.0,30.0,W,White,Seatbelt Violation,Seat belt,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X3


In [21]:
dff = ri[ri['driver_gender'] == 'F']

In [22]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 131138 entries, 2005-02-24 01:20:00 to 2015-12-31 22:39:00
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  131138 non-null  object 
 1   stop_date           131138 non-null  object 
 2   stop_time           131138 non-null  object 
 3   location_raw        131138 non-null  object 
 4   police_department   131138 non-null  object 
 5   driver_gender       131138 non-null  object 
 6   driver_age_raw      131137 non-null  float64
 7   driver_age          130828 non-null  float64
 8   driver_race_raw     131138 non-null  object 
 9   driver_race         131138 non-null  object 
 10  violation_raw       131138 non-null  object 
 11  violation           131138 non-null  object 
 12  search_conducted    131138 non-null  object 
 13  search_type_raw     2459 non-null    object 
 14  search_type         2459 non-null    object 
 15  

In [23]:
dfm = ri[ri['driver_gender'] == 'M']

In [24]:
violation_F = dff.groupby('violation').size()
violation_F

violation
Equipment              14039
Moving violation       17911
Other                   3791
Registration/plates     5649
Seat belt               3550
Speeding               86198
dtype: int64

In [25]:
len(dff)

131138

In [26]:
(violation_F/len(dff)) * 100

violation
Equipment              10.705516
Moving violation       13.658131
Other                   2.890848
Registration/plates     4.307676
Seat belt               2.707072
Speeding               65.730757
dtype: float64

In [27]:
dfm = ri[ri['driver_gender'] == 'M']

In [40]:
dfm.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 349446 entries, 2005-01-02 01:55:00 to 2015-12-31 23:48:00
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  349446 non-null  object 
 1   stop_date           349446 non-null  object 
 2   stop_time           349446 non-null  object 
 3   location_raw        349446 non-null  object 
 4   police_department   349446 non-null  object 
 5   driver_gender       349446 non-null  object 
 6   driver_age_raw      349446 non-null  float64
 7   driver_age          348118 non-null  float64
 8   driver_race_raw     349446 non-null  object 
 9   driver_race         349446 non-null  object 
 10  violation_raw       349446 non-null  object 
 11  violation           349446 non-null  object 
 12  search_conducted    349446 non-null  object 
 13  search_type_raw     15303 non-null   object 
 14  search_type         15303 non-null   object 
 15  

In [28]:
violation_M = dfm.groupby('violation').size()
violation_M

violation
Equipment               47211
Moving violation        72317
Other                   20425
Registration/plates     14181
Seat belt               12774
Speeding               182538
dtype: int64

In [29]:
(violation_M/len(dfm)) * 100

violation
Equipment              13.510242
Moving violation       20.694757
Other                   5.844966
Registration/plates     4.058138
Seat belt               3.655500
Speeding               52.236397
dtype: float64

***

## Comparing speeding outcomes by gender

When a driver is pulled over for speeding, many people believe that gender has an impact on whether the driver will receive a ticket or a warning. Can you find evidence of this in the dataset?

First, you'll create two ``DataFrames`` of drivers who were stopped for ``speeding``: one containing ***females*** and the other containing ***males***.

Then, for each **gender**, you'll use the ``stop_outcome`` column to calculate what percentage of stops resulted in a ``"Citation"`` (meaning a ticket) versus a ``"Warning"``.

**INSTRUCTIONS**

*   Create a ``DataFrame``, ``female_and_speeding``, that only includes female drivers who were stopped for speeding.
*   Create a ``DataFrame``, ``male_and_speeding``, that only includes male drivers who were stopped for speeding.
*   Count the **stop outcomes** for the female drivers and express them as proportions.
*   Count the **stop outcomes** for the male drivers and express them as proportions.

In [30]:
female_and_speeding = dff[dff['violation'] == 'Speeding']

In [31]:
male_and_speeding = dfm[dfm['violation'] == 'Speeding']

In [32]:
female_and_speeding['stop_outcome'].isnull().sum()

0

In [33]:
female_and_speeding['stop_outcome'].value_counts()

Citation            82168
Warning              3362
Arrest Driver         456
Arrest Passenger       89
N/D                    78
No Action              45
Name: stop_outcome, dtype: int64

In [34]:
(female_and_speeding['stop_outcome'].value_counts()/len(female_and_speeding['stop_outcome'])) * 100

Citation            95.324718
Warning              3.900323
Arrest Driver        0.529015
Arrest Passenger     0.103251
N/D                  0.090489
No Action            0.052205
Name: stop_outcome, dtype: float64

In [35]:
male_and_speeding['stop_outcome'].value_counts()

Citation            172432
Warning               6587
Arrest Driver         2878
Arrest Passenger       231
N/D                    216
No Action              194
Name: stop_outcome, dtype: int64

In [36]:
(male_and_speeding['stop_outcome'].value_counts()/len(male_and_speeding['stop_outcome'])) * 100

Citation            94.463619
Warning              3.608564
Arrest Driver        1.576658
Arrest Passenger     0.126549
N/D                  0.118332
No Action            0.106279
Name: stop_outcome, dtype: float64

***

## Calculating the search rate

During a traffic stop, the police officer sometimes conducts a search of the vehicle. In this exercise, you'll calculate the percentage of all stops that result in a vehicle search, also known as the **search rate**.

**INSTRUCTIONS**

*   Check the data type of ``search_conducted`` to confirm that it's a ``Boolean Series``.
*   Calculate the search rate by counting the ``Series`` values and expressing them as proportions.
*   Calculate the search rate by taking the mean of the ``Series``. (It should match the proportion of ``True`` values calculated above.)

In [37]:
ri['search_conducted'].head()

stop_datetime
2005-01-02 01:55:00    False
2005-01-02 20:30:00    False
2005-01-04 12:55:00    False
2005-01-06 01:30:00    False
2005-01-12 08:05:00    False
Name: search_conducted, dtype: object

In [38]:
(ri['search_conducted'].value_counts()/len(ri))* 100

False    96.30408
True      3.69592
Name: search_conducted, dtype: float64

In [39]:
ri['search_conducted'].mean()*100

3.6959199640437466

***

## Comparing search rates by gender

You'll compare the rates at which **female** and **male** drivers are searched during a traffic stop. Remember that the vehicle search rate across all stops is about **3.8%**.

First, you'll filter the ``DataFrame`` by gender and calculate the search rate for each group separately. Then, you'll perform the same calculation for both genders at once using a ``.groupby()``.

**INSTRUCTIONS 1/3**

*   Filter the ``DataFrame`` to only include **female** drivers, and then calculate the search rate by taking the mean of ``search_conducted``.

In [40]:
female_search_rate = dff['search_conducted'].mean()
female_search_rate

0.018751239152648355

In [41]:
dff['search_conducted']

stop_datetime
2005-02-24 01:20:00    False
2005-03-14 10:00:00    False
2005-03-29 23:20:00    False
2005-06-06 13:20:00    False
2005-06-18 16:30:00    False
                       ...  
2015-12-31 21:59:00    False
2015-12-31 22:09:00    False
2015-12-31 22:10:00    False
2015-12-31 22:10:00    False
2015-12-31 22:39:00    False
Name: search_conducted, Length: 131138, dtype: object

**INSTRUCTIONS 2/3**

*   Filter the ``DataFrame`` to only include **male** drivers, and then repeat the search rate calculation.

In [42]:
male_search_rate = dfm['search_conducted'].mean()
male_search_rate

0.04379217389811301

**INSTRUCTIONS 3/3**

*   Group by driver gender to calculate the search rate for both groups simultaneously. (It should match the previous results.)

In [43]:
ri.columns

Index(['id', 'stop_date', 'stop_time', 'location_raw', 'police_department',
       'driver_gender', 'driver_age_raw', 'driver_age', 'driver_race_raw',
       'driver_race', 'violation_raw', 'violation', 'search_conducted',
       'search_type_raw', 'search_type', 'contraband_found', 'stop_outcome',
       'is_arrested', 'stop_duration', 'out_of_state', 'drugs_related_stop',
       'district'],
      dtype='object')

In [44]:
ri.groupby(['driver_gender', 'search_conducted']).agg({'search_conducted':'count'}).groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

search_conducted
driver_gender search_conducted                  
F             False                    98.124876
              True                      1.875124
M             False                    95.620783
              True                      4.379217

***

## Adding a second factor to the analysis

Even though the search rate for males is much higher than for females, it's possible that the difference is mostly due to a second factor.

For example, you might hypothesize that the search rate varies by violation type, and the difference in search rate between males and females is because they tend to commit different violations.

You can test this hypothesis by examining the search rate for each combination of gender and violation. If the hypothesis was true, you would find that males and females are searched at about the same rate for each violation. Find out below if that's the case!

**INSTRUCTIONS 1/2**

*   Use a ``.groupby()`` to calculate the search rate for each combination of gender and violation. Are males and females searched at about the same rate for each violation?

In [45]:
dff.groupby(['violation','search_conducted']).agg({'search_conducted':'count'}).groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

search_conducted
violation           search_conducted                  
Equipment           False                    95.975497
                    True                      4.024503
Moving violation    False                    96.197867
                    True                      3.802133
Other               False                    95.410182
                    True                      4.589818
Registration/plates False                    94.530005
                    True                      5.469995
Seat belt           False                    98.225352
                    True                      1.774648
Speeding            False                    99.226200
                    True                      0.773800

In [46]:
dfm.groupby(['violation','search_conducted']).agg({'search_conducted':'count'}).groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))

search_conducted
violation           search_conducted                  
Equipment           False                    92.908432
                    True                      7.091568
Moving violation    False                    94.084379
                    True                      5.915621
Other               False                    95.388005
                    True                      4.611995
Registration/plates False                    89.641069
                    True                     10.358931
Seat belt           False                    96.829497
                    True                      3.170503
Speeding            False                    97.336993
                    True                      2.663007

**INSTRUCTIONS 2/2**

*   Reverse the ordering to group by violation before gender. The results may be easier to compare when presented this way.

In [47]:
ri.groupby(['driver_gender','violation','search_conducted']).agg({'search_conducted':'count'}).groupby(level=(0,1)).apply(lambda x:
                                                 100 * x / float(x.sum()))

search_conducted
driver_gender violation           search_conducted                  
F             Equipment           False                    95.975497
                                  True                      4.024503
              Moving violation    False                    96.197867
                                  True                      3.802133
              Other               False                    95.410182
                                  True                      4.589818
              Registration/plates False                    94.530005
                                  True                      5.469995
              Seat belt           False                    98.225352
                                  True                      1.774648
              Speeding            False                    99.226200
                                  True                      0.773800
M             Equipment           False                    92.908432
                                  True                      7.091568
              Moving violation    False                    94.084379
                                  True                      5.915621
              Other               False                    95.388005
                                  True                      4.611995
              Registration/plates False                    89.641069
                                  True                     10.358931
              Seat belt           False                    96.829497
                                  True                      3.170503
              Speeding            False                    97.336993
                                  True                      2.663007

In [48]:
ri.groupby(['violation','driver_gender','search_conducted']).agg({'search_conducted':'count'}).groupby(level=(0,1)).apply(lambda x:
                                                 100 * x / float(x.sum()))

search_conducted
violation           driver_gender search_conducted                  
Equipment           F             False                    95.975497
                                  True                      4.024503
                    M             False                    92.908432
                                  True                      7.091568
Moving violation    F             False                    96.197867
                                  True                      3.802133
                    M             False                    94.084379
                                  True                      5.915621
Other               F             False                    95.410182
                                  True                      4.589818
                    M             False                    95.388005
                                  True                      4.611995
Registration/plates F             False                    94.530005
                                  True                      5.469995
                    M             False                    89.641069
                                  True                     10.358931
Seat belt           F             False                    98.225352
                                  True                      1.774648
                    M             False                    96.829497
                                  True                      3.170503
Speeding            F             False                    99.226200
                                  True                      0.773800
                    M             False                    97.336993
                                  True                      2.663007

***

## Counting protective frisks

During a vehicle search, the police officer may pat down the driver to check if they have a weapon. This is known as a ``"protective frisk."``

You'll first check to see how many times "Protective Frisk" was the only search type. Then, you'll use a string method to locate all instances in which the driver was frisked.

**INSTRUCTIONS**

*   Count the ``search_type`` values to see how many times ``"Protective Frisk"`` was the only search type.
*   Create a new column, frisk, that is ``True`` if ``search_type`` contains the string ``"Protective Frisk"`` and ``False`` otherwise.
*   Check the data type of frisk to confirm that it's a ``Boolean Series``.
*   Take the sum of frisk to count the total number of frisks.

In [49]:
ri['search_type'].value_counts()

Incident to Arrest                                          6998
Probable Cause                                              4989
Reasonable Suspicion                                        1141
Inventory                                                   1101
Protective Frisk                                             879
Incident to Arrest,Inventory                                 649
Incident to Arrest,Probable Cause                            552
Probable Cause,Reasonable Suspicion                          334
Probable Cause,Protective Frisk                              221
Incident to Arrest,Protective Frisk                          158
Incident to Arrest,Inventory,Probable Cause                  151
Inventory,Probable Cause                                     132
Protective Frisk,Reasonable Suspicion                         83
Incident to Arrest,Inventory,Protective Frisk                 77
Incident to Arrest,Probable Cause,Protective Frisk            74
Inventory,Protective Fris

In [54]:
ri['frisk'] = ri['search_type'].str.lower().str.contains('protective frisk',na=False, regex=True)

In [55]:
ri['frisk'].value_counts()

False    478986
True       1598
Name: frisk, dtype: int64

***

## Comparing frisk rates by gender

You'll compare the rates at which female and male drivers are frisked during a search. Are males frisked more often than females, perhaps because police officers consider them to be higher risk?

Before doing any calculations, it's important to filter the ``DataFrame`` to only include the relevant subset of data, namely stops in which a search was conducted.

**INSTRUCTIONS**

*   Create a ``DataFrame``, searched, that only contains rows in which ``search_conducted`` is ``True``.
*   Take the mean of the frisk column to find out what percentage of searches included a frisk.
*   Calculate the frisk rate for each gender using a ``.groupby()``.

In [93]:
ri.head()

,id,stop_date,stop_time,location_raw,police_department,driver_gender,driver_age_raw,driver_age,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district,frisk
stop_datetime,,,,,,,,,,,,,,,,,,,,,,,
2005-01-02 01:55:00,RI-2005-00001,2005-01-02,01:55,Zone K1,600,M,1985.0,20.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone K1,False
2005-01-02 20:30:00,RI-2005-00002,2005-01-02,20:30,Zone X4,500,M,1987.0,18.0,W,White,Speeding,Speeding,False,NaN,NaN,False,Citation,False,16-30 Min,False,False,Zone X4,False
2005-01-04 12:55:00,RI-2005-00004,2005-01-04,12:55,Zone X4,500,M,1986.0,19.0,W,White,Equipment/Inspection Violation,Equipment,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4,False
2005-01-06 01:30:00,RI-2005-00005,2005-01-06,01:30,Zone X4,500,M,1978.0,27.0,B,Black,Equipment/Inspection Violation,Equipment,False,NaN,NaN,False,Citation,False,0-15 Min,False,False,Zone X4,False
2005-01-12 08:05:00,RI-2005-00006,2005-01-12,08:05,Zone X1,000,M,1973.0,32.0,B,Black,Call for Service,Other,False,NaN,NaN,False,Citation,False,30+ Min,True,False,Zone X1,False


In [63]:
searched = ri[ri['search_conducted'] == True]

In [64]:
searched

,id,stop_date,stop_time,location_raw,police_department,driver_gender,driver_age_raw,driver_age,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,stop_duration,out_of_state,drugs_related_stop,district,frisk
stop_datetime,,,,,,,,,,,,,,,,,,,,,,,
2005-01-24 20:32:00,RI-2005-00010,2005-01-24,20:32,Zone K1,600,M,1987.0,18.0,W,White,Speeding,Speeding,True,Probable Cause,Probable Cause,True,Citation,False,0-15 Min,True,True,Zone K1,False
2005-02-09 03:05:00,RI-2005-00011,2005-02-09,03:05,Zone X4,500,M,1976.0,29.0,W,White,Registration Violation,Registration/plates,True,"Probable Cause,Terry Frisk","Probable Cause,Protective Frisk",False,Citation,False,0-15 Min,False,False,Zone X4,True
2005-08-28 01:00:00,RI-2005-00084,2005-08-28,01:00,Zone X1,000,M,1979.0,26.0,W,White,Other Traffic Violation,Moving violation,True,"Incident to Arrest,Terry Frisk","Incident to Arrest,Protective Frisk",False,Arrest Driver,True,16-30 Min,True,False,Zone X1,True
2005-09-15 02:20:00,RI-2005-00094,2005-09-15,02:20,Zone X4,500,M,1988.0,17.0,W,White,Other Traffic Violation,Moving violation,True,Incident to Arrest,Incident to Arrest,False,Arrest Driver,True,16-30 Min,False,False,Zone X4,False
2005-09-24 02:20:00,RI-2005-00115,2005-09-24,02:20,Zone K3,300,M,1987.0,18.0,W,White,Other Traffic Violation,Moving violation,True,Incident to Arrest,Incident to Arrest,False,Arrest Driver,True,16-30 Min,False,False,Zone K3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-28 11:05:00,RI-2015-46706,2015-12-28,11:05,Zone K3,300,F,1996.0,19.0,W,White,APB,Other,True,Incident to Arrest,Incident to Arrest,True,Citation,False,16-30 Min,True,True,Zone K3,False
2015-12-30 01:37:00,RI-2015-46870,2015-12-30,01:37,Zone X4,500,M,1979.0,36.0,W,White,Registration Violation,Registration/plates,True,Terry Frisk,Protective Frisk,False,Citation,False,0-15 Min,False,False,Zone X4,True
2015-12-30 08:51:00,RI-2015-46903,2015-12-30,08:51,Zone K3,300,F,1987.0,28.0,H,Hispanic,Speeding,Speeding,True,"Probable Cause,Reasonable Suspicion","Probable Cause,Reasonable Suspicion",True,Citation,False,30+ Min,True,True,Zone K3,False


In [57]:
ri['search_conducted'].value_counts()

False    462822
True      17762
Name: search_conducted, dtype: int64

In [65]:
searched.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17762 entries, 2005-01-24 20:32:00 to 2015-12-30 15:58:00
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  17762 non-null  object 
 1   stop_date           17762 non-null  object 
 2   stop_time           17762 non-null  object 
 3   location_raw        17762 non-null  object 
 4   police_department   17762 non-null  object 
 5   driver_gender       17762 non-null  object 
 6   driver_age_raw      17762 non-null  float64
 7   driver_age          17750 non-null  float64
 8   driver_race_raw     17762 non-null  object 
 9   driver_race         17762 non-null  object 
 10  violation_raw       17762 non-null  object 
 11  violation           17762 non-null  object 
 12  search_conducted    17762 non-null  object 
 13  search_type_raw     17762 non-null  object 
 14  search_type         17762 non-null  object 
 15  contraband_found  

In [68]:
searched['frisk'].value_counts(normalize=True)

False    0.910033
True     0.089967
Name: frisk, dtype: float64

In [73]:
searched.groupby('driver_gender')['frisk'].value_counts()

driver_gender  frisk
F              False     2295
               True       164
M              False    13869
               True      1434
Name: frisk, dtype: int64

In [72]:
searched.groupby('driver_gender')['frisk'].value_counts(normalize=True)

driver_gender  frisk
F              False    0.933306
               True     0.066694
M              False    0.906293
               True     0.093707
Name: frisk, dtype: float64